In [3]:
import os
import pickle
import numpy as np

os.chdir(os.getcwd() + "/../../")
from llm_earnings_call import DATA_PATH
from llm_earnings_call.interfaces import CFOSearcher, ChatGPTPrompter
from llm_earnings_call.settings import Settings 

Load data

In [4]:
settings = Settings()
# Open the pickle file for reading in binary mode
with open(DATA_PATH / 'main_df_earnings.pkl', 'rb') as file:
    # Load the data from the pickle file
    df = pickle.load(file)

Obtain CFO interventions

In [5]:
df = df.loc[~df['content_earnings_call'].isnull()]
df = df.iloc[0:3,]
display(df)

,ticker,date,close,volume,earnings_date,quarter,year,content_earnings_call
32,A,2020-02-19,85.349998,4741300.0,2020-05-29,1.0,2020.0,Operator: Good afternoon and welcome to the Ag...
98,A,2020-05-22,84.980003,5063100.0,2020-09-01,2.0,2020.0,"Operator: Good afternoon, and welcome to the A..."
159,A,2020-08-19,97.959999,2830800.0,2020-12-17,3.0,2020.0,"Operator: Good afternoon, and welcome to the A..."


In [6]:
cfo = CFOSearcher(df, 'content_earnings_call')
cfo.get_cfo_info()
df_cfo = cfo.get_full_cfo_interventions()

/Users/ericmatamoros/Desktop/llm_earnings_call/llm_earnings_call/interfaces/CFOSearcher.py:119: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = self._df.groupby(['ticker', 'date']).apply(lambda x: get_cfo_intervention(x, self._earnings_col, 'cfo'))


In [7]:
display(df_cfo)

,ticker,date,close,volume,earnings_date,quarter,year,content_earnings_call,cfo,cfo_interventions
32,A,2020-02-19,85.349998,4741300.0,2020-05-29,1.0,2020.0,Operator: Good afternoon and welcome to the Ag...,Bob McMahon,Bob McMahon: Thank you Mike and good afternoon...
98,A,2020-05-22,84.980003,5063100.0,2020-09-01,2.0,2020.0,"Operator: Good afternoon, and welcome to the A...",Bob McMahon,"Bob McMahon: Thank you, Mike, and good afterno..."
159,A,2020-08-19,97.959999,2830800.0,2020-12-17,3.0,2020.0,"Operator: Good afternoon, and welcome to the A...",Bob McMahon,"Bob McMahon: Thank you, Mike, and good afterno..."


Use Large Language Model to retrieve financial information

In [9]:
llm = ChatGPTPrompter(settings.openai_api, 'gpt-3.5-turbo-16k')
response = llm.obtain_earnings_call_info(df_cfo.ticker.values[0], df_cfo.cfo_interventions.values[0])

In [ ]:
response